### **4.10 - [Práctica] Organizing your DAGs folder**

#### **`Package a DAG in a ZIP File`**

Empecemos por el que utiliza los archivos zip. Trabajaremos con el siguiente código:

<center><img src="https://i.postimg.cc/P5hhGZRL/a692.png"></center>

¿Cómo podemos modificar este DAG para empaquetarlo en un archivo zip y hacer el código más limpio? 

Una cosa que podríamos hacer es eliminar las tres funciones de python del código del DAG y ponerlas en un archivo externo que importaremos si es necesario. Así, en la carpeta dags, creamos una nueva carpeta con el nombre "functions" por ejemplo, y creamos un nuevo archivo python en ella llamado helpers.py, así. Luego, copiamos y eliminamos las tres funciones python del DAG packaged_dag, y las pegamos en helpers.py. Guardamos el archivo.

<center><img src="https://i.postimg.cc/JhzL08mM/a693.png"></center>

Además, otro archivo python que debemos crear es **`__init__.py`**. El archivo **`__init__.py`** hace que Python trate los directorios que lo contienen como módulos. También es el primer archivo que se carga en un módulo, por lo que se puede utilizar para ejecutar el código que se quiera ejecutar cada vez que se cargue un módulo. Básicamente, sin este archivo, no podríamos importar nuestras funciones del DAG. Notar que el archivo quedará vacío.

<center><img src="https://i.postimg.cc/fRKsMLG8/a694.png"></center>

Bien, volvamos a nuestro DAG e importemos las funciones de helpers:

<center><img src="https://i.postimg.cc/cHKG7crz/a695.png"></center>

Ahora que hemos externalizado las funciones de python y hemos limpiado el código, lo último que queremos es empaquetar todo lo que hemos creado hasta ahora en un archivo zip. Desde tu terminal, comprueba que estás en la carpeta airflow-materials/airflow-section-4 y escribe "cd mnt/airflow/dags". Enter. Si escribes "ls" podrás ver nuestro dag aquí, así como la carpeta que acabamos de crear.

<center><img src="https://i.postimg.cc/9fgjqkFT/a696.png"></center>

Para empaquetarlos, escribe el comando **`"zip -rm package_dag.zip packaged_dag.py functions/"`**, pulsa enter, y obtendremos nuestro archivo zip. No olvides que es muy importante que tus DAGs estén en la raíz (root) del archivo zip, de lo contrario Airflow no podrá cargarlos.

<center><img src="https://i.postimg.cc/przN4XQY/a697.png"></center>

Bien, ahora volvamos a la carpeta 'airflow-section-4' con el comando "cd -", e iniciemos Airflow con el script start.sh. Una vez que el contenedor Docker se está ejecutando, vaya a su navegador web y abra la interfaz de usuario de Airflow. Como puedes ver aquí, el DAG 'packaged_dag' se ha cargado con éxito desde el archivo zip.

<center><img src="https://i.postimg.cc/59CM6ncY/a698.png"></center>
<center><img src="https://i.postimg.cc/vBpF06nZ/a699.png"></center>

___

#### **`Utilizar DagBags`**

Muy bien, ahora que hemos aprendido cómo empaquetar nuestros DAGs con archivos zip para organizar mejor la carpeta dags, vamos a descubrir cómo hacerlo con DagBags. Antes de volver a tu editor de código, vamos a echar un vistazo a los registros del webserver desde tu terminal. Escribe "docker ps", luego escribe "docker logs -f", copia y pega el id del contenedor y pulsa enter. Si revisas cuidadosamente los logs, verás una línea, como esta, que dice "filling up the DagBag (llenando el DagBag)" con la ruta correspondiente a la carpeta dags de Airflow.

<center><img src="https://i.postimg.cc/FsXXc3mP/a700.png"></center>

Cada vez que se ve esta línea, se crea un nuevo DagBag para empezar a recoger los DAGs de una ruta determinada. Este proceso se activa cada 30 segundos por defecto, pero se puede personalizar cambiando el parámetro "worker_refresh_interval" en el archivo de configuración airflow.cfg. Este parámetro define el número de segundos antes de refrescar los gunicorn workers. Por ahora, tenga en cuenta que cada 30 segundos se carga (filled) un DagBag y por lo tanto si usted agrega un nuevo Dag a su dags, tendrás que esperar como máximo 30 segundos antes de que aparezca en la DAGs view.

Dicho esto, lo que queremos ahora es crear nuevos DagBags buscando en ubicaciones diferentes a la ruta de dag por defecto (default dag path). Para ello, necesitamos crear un pequeño script de python que ya he hecho para ti en la carpeta dags. Antes de volver a su editor de código, detenga Airflow ejecutando el script start.sh. Ahora vuelve a tu editor de código, y abre el archivo add_dagbags.py. Este script crea un DagBag para cada ruta dada de la matriz dags_dirs aquí, y hace que el DagBag esté disponible globalmente. Es importante tener en cuenta que Airflow debe ser capaz de acceder a las rutas dadas aquí, de lo contrario no funcionará. Primero, descomente el script para activarlo y guarde el archivo. A continuación, copie el dag backfill.py, por ejemplo, y péguelo en las dos carpetas project_a y project_b. Observa que esas dos carpetas no están en la carpeta dags. Renombra los dags con project_a y project_b según donde estén. No te olvides de cambiar también el dag_id, y guarda los archivos.

<center><img src="https://i.postimg.cc/QxgGqCdj/a701.png"></center>
<center><img src="https://i.postimg.cc/T3R8NxKg/a702.png"></center>
<center><img src="https://i.postimg.cc/rwgBvH4b/a703.png"></center>
<center><img src="https://i.postimg.cc/DzZ9Fcdz/a704.png"></center>
<center><img src="https://i.postimg.cc/7LWpLTmn/a705.png"></center>

Vaya al terminal, ejecute Airflow con el script start.sh y vuelva a la interfaz de usuario de Airflow. Aquí están. Perfecto, ahora somos capaces de cargar con éxito dags desde diferentes lugares. Esto puede ser extremadamente útil si usted está trabajando con muchos DAGs y archivos y desea separarlos de acuerdo a los proyectos o equipos de trabajo, por ejemplo. El inconveniente de esta técnica es el hecho de que no podrás ver si hay algún error en tus DAGs desde la UI. Para verificar esto, hagamos un error en el dag project_b. Desde tu editor de código, abre el archivo project_b.py y elimina la importación del BashOperator así. Guarda el archivo y vuelve a la UI. Ahora si empiezas a refrescar la página, no hay ninguna advertencia, ni error, pero si revisas los logs del webserver, en algún momento, verás que el error es bien detectado por Airflow. Así que tenga en cuenta que, si después de cambiar un DAG, se rompe y no has reiniciado el webserver, no podrás ver que ya no funciona desde la UI. Al menos, no obtendrás un error explícito. Por último, no olvides crear un archivo .airflowignore en tu carpeta dags si hay archivos o carpetas que no quieres que Airflow analice.

<center><img src="https://i.postimg.cc/mD1K12nf/a706.png"></center>
<center><img src="https://i.postimg.cc/mkxJYWxc/a707.png"></center>